In [3]:
import os
import pandas as pd
import json
import numpy as np
import random
import gensim
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk import RegexpTokenizer
from gensim.models import Word2Vec
import matplotlib.pyplot as plt

import os
from nltk.corpus import stopwords

stop = stopwords.words('english')
stop+=['?','!','.',',',':',';']

%matplotlib inline
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/williamcottrell72/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/williamcottrell72/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [449]:
followers = pd.read_json('https://s3-us-west-1.amazonaws.com/primebotbucket/Data/08_08_18_followers.json', lines=True)

In [450]:
pages=pd.read_json('https://s3-us-west-1.amazonaws.com/primebotbucket/Data/pages.json',lines=True)

In [4]:
messages1 = pd.read_json('https://s3-us-west-1.amazonaws.com/primebotbucket/Data/08_08_18_messages.json', lines=True)

In [6]:
messages2 = pd.read_json('https://s3-us-west-1.amazonaws.com/primebotbucket/Data/08_08_18_v2messages.json', lines=True)

In [7]:
messages3 = pd.read_json('https://s3-us-west-1.amazonaws.com/primebotbucket/Data/6_25_18_messages.json', lines=True)

In [7]:
m1=messages1['message']
messages=[]
for mes in m1:
    if 'message' in mes.keys():
        try:
            messages.append(mes['message']['text'])
        except:
            pass
    

In [12]:
import pickle as pkl
with open('messages.pkl','wb') as file:
    pkl.dump(messages,file)

In [454]:
subset=np.random.choice(range(len(messages1)),5000)
subset2=np.random.choice(range(len(messages1)),100)
short_messages=messages1.iloc[subset]
short_messages2=messages1.iloc[subset2]

In [453]:
def clean_messages(short_messages):
    short_messages['new_date']=short_messages['date'].apply(lambda x: x['$date']).values
    short_messages['message_id']=short_messages['_id'].apply(lambda x: x['$oid'])

    text=[]
    sender=[]
    timestamp=[]
    recipient=[]
    nlp=[]
    for x in short_messages['message']:    
        sender.append(x['sender']['id'])
        timestamp.append(x['timestamp'])
        recipient.append(x['recipient']['id'])
        if 'postback' in x.keys():
            text.append(x['postback']['title'])
            nlp.append(0)

        else:
            try:
                text.append(x['message']['text'])
                nlp.append(1)
            except:
                text.append('NaN')
                nlp.append(0)

    replace=pd.DataFrame({'text':text,'sender':sender,'recipient':recipient,'timestamp':timestamp,'nlp':nlp})

    sm=short_messages.reset_index().drop(['index','_id','message','date','__v'],axis=1)
    sm_new=sm.join(replace)
    return sm_new


In [455]:
res=clean_messages(short_messages2)

/Users/williamcottrell72/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/williamcottrell72/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [215]:
res[res.nlp==1]

,index,__v,pageId,new_date,message_id,text,sender,recipient,timestamp,nlp
15,103718,0,1.748597e+15,2018-07-27T08:42:41.317Z,5b5adb010caa170014b239c9,HAHAHAHAHA,1801476999944417,1748596732049183,1.532681e+12,1
31,250941,0,1.748597e+15,2018-07-29T02:22:32.828Z,5b5d24e85d30c600141933c5,fuck yiu,1316082558492369,1748596732049183,1.532831e+12,1
36,272599,0,1.748597e+15,2018-07-29T08:50:55.618Z,5b5d7fef5d30c6001419e79a,More pick up lines,1726715647445499,1748596732049183,1.532854e+12,1
46,24930,0,1.748597e+15,2018-07-26T11:28:18.339Z,5b59b052d86a92001459c699,Up your pickup game,1748717728505426,1748596732049183,1.532604e+12,1
51,205044,0,1.748597e+15,2018-07-28T13:00:34.045Z,5b5c68f25d30c6001417b26f,Im sorry,1718344218211401,1748596732049183,1.532783e+12,1
59,33988,0,1.748597e+15,2018-07-26T12:01:45.643Z,5b59b829d86a9200145a0e05,More,1597960256910477,1748596732049183,1.532607e+12,1
61,809565,0,1.553195e+14,2018-08-05T22:11:23.112Z,5b67760b0051520014a5c06c,"YES, It Sure Does!",1729137037134169,155319498535430,1.533507e+12,1
65,108580,0,1.748597e+15,2018-07-27T10:07:04.377Z,5b5aeec80caa170014b2618f,Fj,1334600366642632,1748596732049183,1.532686e+12,1
68,274839,0,1.748597e+15,2018-07-29T09:26:49.609Z,5b5d88595d30c6001419f9e2,Funny T-Shirts,1815624365152088,1748596732049183,1.532856e+12,1
80,45935,0,1.748597e+15,2018-07-26T13:25:27.708Z,5b59cbc70caa170014b0621f,Funny T-shirts,2242475405793416,1748596732049183,1.532612e+12,1


In [460]:
def clean_pages(pages):
    fans=[]
    usr_name=[]
    for p in pages['information']:
        try:
            fans.append(p['fanCount'])
        except:
            fans.append('NaN')
        try:
            usr_name.append(p['username'])
        except:
            usr_name.append('NaN')

    message_id=[]
    for m in pages['_id']:
        try:
            message_id.append(m['$oid'])
        except:
            message_id.append('NaN')

    usr_id=[]
    for m in pages['userId']:
        try:
            usr_id.append(m['$oid'])
        except:
            usr_id.append('NaN')

    bot_id=[]
    for m in pages['botId']:
        try:
            bot_id.append(m['$oid'])
        except:
            bot_id.append('NaN')

    page_id=[]
    for m in pages['pageId']:
        try:
            page_id.append(m['$numberLong'])
        except:
            page_id.append(m)


    new_date=[]
    for m in pages['date']:
        new_date.append(m['$date'])
        
    pages['bot_birthday']=new_date
    pages['usr_id']=usr_id
    pages['message_id']=message_id
    pages['usr_name']=usr_name
    pages['fans']=fans
    pages['page_id']=page_id
    pages['bot_Id']=bot_id
    
    useless=['_id','accessToken','administrators','date','information','__v','fetchedConversations','pageId','insufficientPermissions','userId','whitelistedDomains','insufficientPermissionsLastChecked','tags','botId']
    
    return pages.drop(useless,axis=1)

In [171]:
len(pages)

3845

In [458]:
def clean_followers(pages):
    follower_id=[]
    follower_name=[]
    gender=[]
    for p in pages['profile']:
        try:
            follower_id.append(p['id'])
        except:
            follower_id.append('NaN')
        try:
            follower_name.append(p['last_name']+', '+p['first_name'])
        except:
            follower_name.append('NaN')
        try:
            gender.append(p['gender'])
        except:
            gender.append('NaN')

    message_id=[]
    for m in pages['_id']:
        try:
            message_id.append(m['$oid'])
        except:
            message_id.append('NaN')

#     usr_id=[]
#     for m in pages['userId']:
#         try:
#             usr_id.append(m['$oid'])
#         except:
#             usr_id.append('NaN')

#     bot_id=[]
#     for m in pages['botId']:
#         try:
#             bot_id.append(m['$oid'])
#         except:
#             bot_id.append('NaN')

#     page_id=[]
#     for m in pages['pageId']:
#         try:
#             page_id.append(m['$numberLong'])
#         except:
#             page_id.append(m)


    first_contact=[]
    for m in pages['firstContactDate']:
        first_contact.append(m['$date'])
        
    last_contact=[]
    for m in pages['lastContactDate']:
        last_contact.append(m['$date'])
        
    pages['last_contact']=last_contact
    pages['first_contact']=first_contact
    pages['message_id']=message_id
    pages['follower_name']=follower_name
    pages['follower_id']=follower_id
    pages['gender']=gender
    
    useless=['_id','__v','pageId','userId','firstContactDate','lastContactDate','attributes','profile']
    
    return pages.drop(useless,axis=1)

In [461]:
cf=clean_followers(followers)
cp=clean_pages(pages).drop(['message_id'],axis=1)
#cm=clean_messages(messages1.head(1000))
cm_short=clean_messages(short_messages)

/Users/williamcottrell72/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/williamcottrell72/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [416]:
import pickle as pkl
with open('messages.pkl','wb') as file:
    pkl.dump(messages,file)
    
with open('messages1.pkl','wb') as file:
    pkl.dump(messages1.file)

## F'd Data

1000

In [462]:
send_is_follower=[(x in list(cf['follower_id'])) for x in cm_short['sender']]
send_not_follower=[not x for x in send_is_follower]
send_is_page=[(x in list(cp['page_id']))  for x in cm_short['sender']]
send_is_neither=[(x not in list(cp['page_id'])) and (x not in list(cf['follower_id'])) for x in cm_short['sender']]

rec_is_follower=[(x in list(cf['follower_id'])) for x in cm_short['recipient']]
rec_not_follower=[not x for x in send_is_follower]
rec_is_page=[(x in list(cp['page_id']))  for x in cm_short['recipient']]
rec_is_neither=[(x not in list(cp['page_id'])) and (x not in list(cf['follower_id'])) for x in cm_short['recipient']]


check=[x==y for x,y in zip(send_is_page, send_not_follower)]

In [463]:
#Percentage of entries for which sender has neither a follower_id now a page_id
sum(send_is_neither)/len(send_is_neither)

0.1878

In [464]:
sum(rec_is_neither)/len(rec_is_neither)

0.1912

In [469]:
#Percentage of cases with no id
no_id=[(x and y) for (x,y) in (zip(send_is_neither,rec_is_neither))]
sum(no_id)/len(no_id)

0.1816

In [466]:
len(send_is_follower), len(send_is_neither)


(5000, 5000)

In [468]:
#Percentage of people mimicking robot.
sum([x=='Yes! Another meme!' for x in cm_short[send_is_follower]['text']])/len(cm_short[send_is_follower])

0.7552327013050972

So we see the data is f'd on many levels.

In [433]:
cm

,pageId,new_date,message_id,text,sender,recipient,timestamp,nlp
0,1.748597e+15,2018-07-26T02:08:23.034Z,5b592d17d86a92001458fa99,Fuck you,2626588190700296,1748596732049183,1.532571e+12,1
1,5.275071e+14,2018-07-26T02:08:27.149Z,5b592d1bd86a92001458fa9b,Yes! Another meme!,1706209672761079,527507080600217,1.532571e+12,0
2,5.275071e+14,2018-07-26T02:08:28.613Z,5b592d1cd86a92001458fa9d,Get Started,1732014683531673,527507080600217,1.532571e+12,0
3,5.275071e+14,2018-07-26T02:08:29.284Z,5b592d1dd86a92001458faa0,Yes! Another meme!,1802412613206450,527507080600217,1.532571e+12,0
4,1.748597e+15,2018-07-26T02:08:32.940Z,5b592d20d86a92001458faa2,Sexy,2626588190700296,1748596732049183,1.532571e+12,1
5,5.275071e+14,2018-07-26T02:08:36.849Z,5b592d24d86a92001458faa4,Yes! Another meme!,1706209672761079,527507080600217,1.532571e+12,0
6,5.275071e+14,2018-07-26T02:08:43.314Z,5b592d2bd86a92001458faa6,Get Started,1697548080357440,527507080600217,1.532571e+12,0
7,5.275071e+14,2018-07-26T02:08:51.139Z,5b592d33d86a92001458faa9,Byr,1732014683531673,527507080600217,1.532571e+12,1
8,5.275071e+14,2018-07-26T02:08:54.326Z,5b592d36d86a92001458faab,Bye,1732014683531673,527507080600217,1.532571e+12,1
9,1.748597e+15,2018-07-26T02:08:58.191Z,5b592d3ad86a92001458faad,Doggystyle,2626588190700296,1748596732049183,1.532571e+12,1


In [430]:
sum([(x!=y) for (x,y) in zip(send_is_neither,rec_is_neither)])

7

In [424]:
cm_sf=cm[send_is_follower]
cm_sp=cm[send_is_page]
cm_not_follower=cm[send_not_follower]

In [425]:
# cm_sf_rename=cm_sf.rename({'sender':'follower_id','recipient':'page_id'},axis=1)
# cm_sp_rename=cm_not_follower.rename({'recipient':'follower_id','sender':'non_follower'},axis=1)
cm_fin=pd.concat([cm_sf,cm_not_follower],axis=0,ignore_index=True).drop(['pageId','message_id'],axis=1)

In [427]:
cm_fin['send_is_follower']=send_is_follower
cm_fin[cm_fin.nlp==1].head()

,new_date,text,sender,recipient,timestamp,nlp,send_is_follower
5,2018-07-26T02:08:51.139Z,Byr,1732014683531673,527507080600217,1.532571e+12,1,True
6,2018-07-26T02:08:54.326Z,Bye,1732014683531673,527507080600217,1.532571e+12,1,True
113,2018-07-26T02:13:27.543Z,jmgfhykjykjugjgjigngkhkgjhngb,2285383284812613,527507080600217,1.532571e+12,1,True
118,2018-07-26T02:13:37.918Z,😗lhng,2285383284812613,527507080600217,1.532571e+12,1,True
120,2018-07-26T02:13:46.657Z,jnbkfihbc,2285383284812613,527507080600217,1.532571e+12,1,False


In [344]:
cp_prep=cp.rename({'page_id':'follower_id'},axis=1)

In [345]:
merge2=merge1.merge(cp_prep,on='follower_id',how='left')

In [351]:
merge2['follower_id'][2] in cp['page_id']

False

In [349]:
cp.head()

,botActive,botCreated,name,usr_id,usr_name,fans,bot_Id,page_id,bot_birthday
0,True,True,George Portillo,57e6f48cd053c5000351763d,thegeorgeportillo,3835,591b81ce7252ac0011cd9325,205044679527624,2017-04-16T02:37:06.342Z
1,True,True,Twilight sucks,57e6f48cd053c5000351763d,Twlight.Suckz,101531,58f620f47232890011a39991,3.90676e+10,2017-04-16T02:37:06.345Z
2,True,True,Viral Thread,57e6f48cd053c5000351763d,JungleVT,22672523,58f2d8f3bfa3f800111c47bc,363765800431935,2017-04-16T02:37:06.346Z
3,True,True,Bob,57e6f48cd053c5000351763d,NothingBob,25451,58f2daf4bfa3f800111c47bf,4.78441e+14,2017-04-16T02:37:06.347Z
4,False,False,It was just a dream. Dammit.,57e6f48cd053c5000351763d,thebestpagever,3786921,NaN,2.2508e+11,2017-04-16T02:37:06.350Z


In [346]:
merge2.drop(['subscriptionType'],axis=1)

,follower_id,new_date,nlp,page_id,text,timestamp,last_contact,first_contact,follower_name,gender,botActive,botCreated,name,usr_id,usr_name,fans,bot_Id,bot_birthday
0,1706209672761079,2018-07-26T02:08:27.149Z,0,527507080600217,Yes! Another meme!,1.532571e+12,2018-07-26T14:25:50.027Z,2018-07-22T22:05:11.543Z,"Pena, José Miguel",male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1732014683531673,2018-07-26T02:08:28.613Z,0,527507080600217,Get Started,1.532571e+12,2018-07-26T02:08:54.379Z,2018-07-26T02:08:28.649Z,"Alvarez, Rj",male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1802412613206450,2018-07-26T02:08:29.284Z,0,527507080600217,Yes! Another meme!,1.532571e+12,2018-07-26T02:10:30.424Z,2018-07-18T14:52:58.167Z,"Lopez, Evelyna",female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1706209672761079,2018-07-26T02:08:36.849Z,0,527507080600217,Yes! Another meme!,1.532571e+12,2018-07-26T14:25:50.027Z,2018-07-22T22:05:11.543Z,"Pena, José Miguel",male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1697548080357440,2018-07-26T02:08:43.314Z,0,527507080600217,Get Started,1.532571e+12,2018-07-26T02:08:43.432Z,2018-07-26T02:08:43.335Z,"Rodriguez, Martin",male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1732014683531673,2018-07-26T02:08:51.139Z,1,527507080600217,Byr,1.532571e+12,2018-07-26T02:08:54.379Z,2018-07-26T02:08:28.649Z,"Alvarez, Rj",male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1732014683531673,2018-07-26T02:08:54.326Z,1,527507080600217,Bye,1.532571e+12,2018-07-26T02:08:54.379Z,2018-07-26T02:08:28.649Z,"Alvarez, Rj",male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1706209672761079,2018-07-26T02:09:07.461Z,0,527507080600217,Yes! Another meme!,1.532571e+12,2018-07-26T14:25:50.027Z,2018-07-22T22:05:11.543Z,"Pena, José Miguel",male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1706209672761079,2018-07-26T02:09:14.948Z,0,527507080600217,Yes! Another meme!,1.532571e+12,2018-07-26T14:25:50.027Z,2018-07-22T22:05:11.543Z,"Pena, José Miguel",male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1816751395026861,2018-07-26T02:09:19.829Z,0,527507080600217,Yes! Another meme!,1.532571e+12,2018-08-08T10:24:27.041Z,2018-07-25T14:55:42.125Z,"Kaye, Zyra",female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [242]:
ids=[x for x in followers[ 'follower_id'] if x!='NaN']
cm['recipient'][6] in id_list

True

In [235]:
id_list=list(cf['follower_id'])+list(cp['page_id'])


In [234]:
cp.head()

,botActive,botCreated,name,new_date,usr_id,message_id,usr_name,fans,bot_Id,page_id,bot_birthday
0,True,True,George Portillo,2017-04-16T02:37:06.342Z,57e6f48cd053c5000351763d,58f2d8d2bfa3f800111c478a,thegeorgeportillo,3835,591b81ce7252ac0011cd9325,205044679527624,2017-04-16T02:37:06.342Z
1,True,True,Twilight sucks,2017-04-16T02:37:06.345Z,57e6f48cd053c5000351763d,58f2d8d2bfa3f800111c478b,Twlight.Suckz,101531,58f620f47232890011a39991,3.90676e+10,2017-04-16T02:37:06.345Z
2,True,True,Viral Thread,2017-04-16T02:37:06.346Z,57e6f48cd053c5000351763d,58f2d8d2bfa3f800111c478c,JungleVT,22672523,58f2d8f3bfa3f800111c47bc,363765800431935,2017-04-16T02:37:06.346Z
3,True,True,Bob,2017-04-16T02:37:06.347Z,57e6f48cd053c5000351763d,58f2d8d2bfa3f800111c478d,NothingBob,25451,58f2daf4bfa3f800111c47bf,4.78441e+14,2017-04-16T02:37:06.347Z
4,False,False,It was just a dream. Dammit.,2017-04-16T02:37:06.350Z,57e6f48cd053c5000351763d,58f2d8d2bfa3f800111c478e,thebestpagever,3786921,NaN,2.2508e+11,2017-04-16T02:37:06.350Z
